In [ ]:
from sentence_transformers import SentenceTransformer

from sklearn.metrics.pairwise import cosine_similarity
import sqlite3
import joblib
from tqdm import tqdm
from datetime import datetime

import numpy as np

/Users/magewade/Desktop/ML/bot/lenta_news/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# Загружаем модель
model = SentenceTransformer("sberbank-ai/sbert_large_nlu_ru")


def get_bert_embedding(text):
    # Векторизация текста с использованием модели
    return model.encode(text, convert_to_numpy=True)

In [5]:
def load_news_from_db(
    db_path="/Users/magewade/Desktop/ML/bot/lenta_news/news_database.db",
):
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    cursor.execute("SELECT content FROM articles")
    rows = cursor.fetchall()
    conn.close()
    return [row[0] for row in rows]

In [6]:
# Загружаем новости из базы
news_texts = load_news_from_db()

# Векторизация новостей
news_embeddings = [
    get_bert_embedding(text) for text in tqdm(news_texts, desc="Векторизация новостей")
]

Векторизация новостей: 100%|██████████| 2277/2277 [02:22<00:00, 16.03it/s]


In [7]:
def find_similar_news(query, news_texts, news_vectors, top_k=3):
    # Векторизуем запрос
    query_vec = get_bert_embedding(query).reshape(1, -1)

    # Считаем сходство с новостями
    sims = cosine_similarity(query_vec, news_vectors)[0]

    # Сортируем новости по сходству
    top_indices = sims.argsort()[-top_k:][::-1]
    return [(news_texts[i], sims[i]) for i in top_indices]

In [8]:
joblib.dump((news_texts, news_embeddings), "news_vectors.pkl")

['news_vectors.pkl']

In [9]:
query = "кот"
results = find_similar_news(query, news_texts, news_embeddings)

# Печать результатов
for i, (text, score) in enumerate(results, 1):
    print(f"{i}. ({score:.2f}) {text[:200]}")

1. (0.36) Врач-терапевт и телеведущая Елена Малышева и ее соведущие, кардиолог Герман Гандельман и офтальмолог Михаил Коновалов в эфире программы «Жить здорово!» на Первом канале раскрыли россиянам неожиданную 
2. (0.35) В московской квартире в Северном Измайлово кот по кличке Кузя спас 10-летнего ребенка от пожара. Об этом сообщило МЧС России в своем Telegram-канале. «10-летний Георгий делал уроки, когда его стал хва
3. (0.33) В Новосибирске бездомный кот обрел хозяина и сразу же устроил погром в магазине на тысячи рублей. Об этом сообщает Telegram-канал Mash Siberia. Житель российского города подобрал на улице кота и зашел


In [17]:
def save_embeddings_to_db(db_path, news_texts, embeddings):
    # Соединяемся с базой данных
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    # Для каждой новости и её эмбеддинга добавляем в базу
    for text, embedding in tqdm(
        zip(news_texts, embeddings), desc="Сохранение эмбеддингов в базу"
    ):
        # Преобразуем эмбеддинг в бинарный формат
        embedding_blob = embedding.tobytes()

        # Обновляем эмбеддинг для существующей новости
        cursor.execute(
            """
            UPDATE articles
            SET embedding = ?
            WHERE content = ?
        """,
            (embedding_blob, text),
        )

    # Сохраняем изменения в базе данных
    conn.commit()
    conn.close()

In [13]:
import sqlite3


def add_embedding_column(db_path):
    # Соединяемся с базой данных
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    # Выполняем SQL-команду для добавления колонки
    cursor.execute("ALTER TABLE articles ADD COLUMN embedding BLOB")

    # Сохраняем изменения
    conn.commit()
    conn.close()


# Добавляем колонку в существующую базу данных
add_embedding_column("/Users/magewade/Desktop/ML/bot/lenta_news/news_database.db")

In [18]:
save_embeddings_to_db(
    "/Users/magewade/Desktop/ML/bot/lenta_news/news_database.db",
    news_texts,
    news_embeddings,
)

Сохранение эмбеддингов в базу: 2277it [00:02, 917.96it/s]
